# Assignment 10

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

Naive Bayes classifier assume that the effect of the value of a predictor (x) on a given class (c) is independent of the values of other predictors. This assumption is called class conditional independence. P(c|x) is the posterior probability of class (target) given predictor (attribute).

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

In Bayesian statistics, a maximum a posterior probability (MAP) estimate is an estimate of an unknown quantity, that equals the mode of the posterior distribution. The MAP can be used to obtain a point estimate of an unobserved quantity on the basis of empirical data. It is closely related to the method of maximum likelihood (ML) estimation, but employs an augmented optimization objective which incorporates a prior distribution (that quantifies the additional information available through prior knowledge of a related event) over the quantity one wants to estimate. MAP estimation can therefore be seen as a regularization of ML estimation.
MAP for short is a Bayesian-based approach to estimating a distribution and model parameters that best explain an observed dataset.

In statistics, maximum likelihood estimation (MLE) is a method of estimating the parameters of a probability distribution by maximizing a likelihood function, so that under the assumed statistical model the observed data is most probable. The point in the parameter space that maximizes the likelihood function is called the maximum likelihood estimate. The logic of maximum likelihood is both intuitive and flexible, and as such the method has become a dominant means of statistical inference.

#### 3. What is support vector in SVM?

Support vectors are data points that are closer to the hyperplane and influence the position and orientation of the hyperplane. Using these support vectors, we maximize the margin of the classifier. Deleting the support vectors will change the position of the hyperplane. These are the points that help us build our SVM.

#### 4. What is the intuition behind SVM ?

The SVM, in particular, is looking for a boundary that is maximally far away from any data point. This distance from that decision boundary to the closest data points determines the margin of the classifier.

#### 5. Shortly describ what 'random' means in random forest ?

searching over a random subset of the available decisions when splitting a node, in the context of growing a single tree. 

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

he algorithm first proposes candidate splitting points according to percentiles of feature distribution. The algorithm then maps the continuous features into buckets split by these candidate points, aggregates the statistics and finds the best solution among proposals based on the aggregated statistics.

I believe that this means that candidate splits are proposed for every single feature in the dataset and that all of these candidate splits are inspected to find the optimal one. This means that the method of finding splits really is an approximate form of the algorithm that inspects every single feature and every single split to find the best one but that the approximation only happens at the level of the splits proposed for each feature. Every single feature is still inspected.

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

In [0]:
import pandas as pd
import numpy as np

In [0]:
df =pd.read_csv("/content/drive/My Drive/csv/xinhua.csv",encoding='gb18030')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
id         89611 non-null int64
author     79396 non-null object
source     89609 non-null object
content    87054 non-null object
feature    89611 non-null object
title      89577 non-null object
url        87144 non-null object
dtypes: int64(1), object(6)
memory usage: 4.8+ MB


In [0]:
content=df['content'].tolist()

In [0]:
content[0]

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n'

In [17]:
df['content'].astype('object')
df['source'].astype('object')

0        快科技@http://www.kkj.cn/
1        快科技@http://www.kkj.cn/
2        快科技@http://www.kkj.cn/
3                           新华社
4                         深圳大件事
                  ...          
89606                       新华社
89607                       新华社
89608                       新华社
89609                       新华社
89610                       新华社
Name: source, Length: 89611, dtype: object

In [0]:
def function(s):
    if '新华社' in str(s):
        return 1
    else:
        return 0

In [0]:
type(df.source[0])

str

In [0]:
df['label'] = df['source'].apply(function)

In [0]:
import jieba
stopword =[]
with open('/content/drive/My Drive/Colab Notebooks/stopwords.txt','r') as st:
  for line in st:
    stopword.append(line.strip())

In [0]:
def cut(x):
    segs = []
    for word in jieba.cut(str(x)):
        if word not in stopword:
            segs.append(word)
    return ' '.join(segs)

In [22]:
df['content']= df['content'].apply(cut)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.599 seconds.
Prefix dict has been built successfully.


In [0]:
df.to_csv('/content/drive/My Drive/csv/xinhua_cut.csv')

In [0]:
df=pd.read_csv('/content/drive/My Drive/csv/xinhua_cut.csv')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 9 columns):
Unnamed: 0    89611 non-null int64
id            89611 non-null int64
author        79396 non-null object
source        89609 non-null object
content       87054 non-null object
feature       89611 non-null object
title         89577 non-null object
url           87144 non-null object
label         89611 non-null int64
dtypes: int64(3), object(6)
memory usage: 6.2+ MB


In [3]:
!pip install flair

In [4]:
import torch
%tensorflow_version 2.x
from flair.embeddings import FlairEmbeddings, DocumentPoolEmbeddings, Sentence, BertEmbeddings,WordEmbeddings
# initialise embedding classes
#CUDA_LAUNCH_BLOCKING=1
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')
#bert_embedding = BertEmbeddings('bert-base-chinese')
glove_embedding = WordEmbeddings('glove')
# combine word embedding models
document_embeddings = DocumentPoolEmbeddings([glove_embedding, flair_embedding_backward, flair_embedding_forward])

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#del(X)
#torch.cuda.empty_cache()

In [0]:
CUDA_LAUNCH_BLOCKING=1 
# set up empty tensor
X = torch.empty(size=(len(df.index),4196)).cuda()
#torch.backends.cudnn.enabled = False
# fill tensor with embeddings
i=0

In [0]:
from tqdm import tqdm

In [7]:
for text in tqdm(df['content']):
    sentence = Sentence(str(text))
    document_embeddings.embed(sentence)
    embedding = sentence.get_embedding()
    X[i] = embedding
    i += 1

  1%|          | 607/89611 [11:12<26:30:34,  1.07s/it]

RuntimeError: ignored

# 在训练句向量时CUDA out of memory, 时间不够了，没解决



In [0]:
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC

In [0]:
cls = SVC()
cls.fit(X,Y)

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).